In [121]:
import re
import sys
import gc
import logging
import traceback
import datetime
import numpy as np
import pandas as pd
import bs4
from bs4 import BeautifulSoup as soup
from bs4 import Comment
from urllib.request import urlopen as uReq
from references_dict import Team_Dictionary

In [122]:
# get soup object from link
def get_soup(link):
    uClient = uReq(link)
    page_html = uClient.read()
    uClient.close()
    page_soup = soup(page_html, "lxml")
    return page_soup

# get table from page
# utility method for parsing game page tables
def get_data(page_soup,id,commented=0):
    data = page_soup.find("div",{"id":id})
    if commented > 0:
        comment = data.find(string=lambda text:isinstance(text,Comment))
        data = soup(comment,"lxml")
    players = [tr.find("th",{"scope":"row"}) for tr in data.findAll("tr",{"class":None})]
    stats = [tr.findAll("td") for tr in data.findAll("tr",{"class":None})]
    players.pop(0)
    stats.pop(0)
    return players,stats

In [123]:
season = 2018
week = 4
link = "https://www.pro-football-reference.com/years/"+str(season)+"/week_"+str(week)+".htm"

page_soup = get_soup(link)
games = page_soup.findAll("td",{"class":"gamelink"})

In [128]:
def get_gameinfo(game):
    # get date, page soup for game link
    gameid = (str(game.a['href']))
    date = gameid[11:20]
    link = "https://www.pro-football-reference.com"+gameid
    print(link)
    gameinfo = np.array([gameid,date])
    page_soup = get_soup(link)
    
    # get team names
    gameteams = page_soup.findAll("a",{"itemprop":"name"})
    team_home = gameteams[0].text
    team_away = gameteams[1].text
    
    # get vegas odds
    gameinfo = page_soup.find("div",{"id":"all_game_info"})
    comment = gameinfo.find(string=lambda text:isinstance(text,Comment))
    gameinfo = soup(comment,"lxml")
    gameinfo = [tr.findAll("td") for tr in gameinfo.findAll("tr",{"class":None})]
    vegasline = gameinfo[-2][0].text.strip()
    vegasline = re.split("\s-",vegasline)
    home_fav = 1
    if vegasline[0] == team_away:
        home_fav = 0
    vegasline = float(vegasline[1])
    overunder = float(gameinfo[-1][0].text.split(" ")[0].strip())
    
    # get score
    score = page_soup.findAll("div",{"class":"score"})
    points_home = float(score[0].text.strip())
    points_away = float(score[1].text.strip())
    home_score_diff = points_home - points_away
    
    # calculate vegas results
    ats_result = 0
    if((home_score_diff > vegasline and home_fav == 1) or (home_score_diff < vegasline and home_fav == 0)):
        ats_result = 1
    elif((home_score_diff < vegasline and home_fav == 1) or (home_score_diff > vegasline and home_fav == 0)):
        ats_result = -1   
        
    ou_result = 0
    if(points_home+points_away > overunder):
        ou_result = 1
    elif(points_home+points_away < overunder):
        ou_result = -1
    
    # return metrics in numpy array
    stats = np.array([gameid,date,team_home,points_home,team_away,points_away,home_fav,vegasline,overunder,ats_result,ou_result])
    print(stats)
    return stats

In [129]:
game1 = get_gameinfo(games[0])
game2 = get_gameinfo(games[1])
mygames = [np.array(['gameid','date','team_home','points_home','team_away','points_away','home_fav','vegasline','overunder','ats_result','ou_result'])]
mygames.append(game1)
mygames.append(game2)

https://www.pro-football-reference.com/boxscores/201809270ram.htm
['/boxscores/201809270ram.htm' '201809270' 'Los Angeles Rams' '38.0'
 'Minnesota Vikings' '31.0' '1' '7.5' '48.5' '-1' '1']
https://www.pro-football-reference.com/boxscores/201809300clt.htm
['/boxscores/201809300clt.htm' '201809300' 'Indianapolis Colts' '34.0'
 'Houston Texans' '37.0' '0' '1.0' '48.0' '1' '1']


In [130]:
df = np.vstack(mygames)

In [131]:
df = pd.DataFrame(data=df[1:,1:],index=df[1:,0],columns=df[0,1:])

In [132]:
df

,date,team_home,points_home,team_away,points_away,home_fav,vegasline,overunder,ats_result,ou_result
/boxscores/201809270ram.htm,201809270,Los Angeles Rams,38.0,Minnesota Vikings,31.0,1,7.5,48.5,-1,1
/boxscores/201809300clt.htm,201809300,Indianapolis Colts,34.0,Houston Texans,37.0,0,1.0,48.0,1,1
